This currently does not function, in the middle of a rewrite

In [9]:
# Import PyTorch
import torch
# Get pandas for data manipulation
import pandas as pd
# Import nltk for text processing
import nltk
# Import os for file manipulation
import os
# Import train_test_split from sklearn
from sklearn.model_selection import train_test_split

In [2]:
# Check if CUDA is available
if torch.cuda.is_available():
    print("Using CUDA")
else:
    print("CUDA is not available. Using CPU")

Using CUDA


In [3]:
# Load the data from Datasets/onehotplotgenre.csv
data = pd.read_csv("Datasets/onehotplotgenre.csv")
# Get the first 5 rows of the data
print(data.head())

                                    Title  \
0          #1 Cheerleader Camp (2010) (V)   
1                 #1 Serial Killer (2013)   
2  #1 at the Apocalypse Box Office (2015)   
3                             #137 (2011)   
4                              #30 (2013)   

                                                Plot  Action  Adventure  \
0  When they're hired to work at a cheerleading c...       0          0   
1  Years of seething rage against the racism he's...       0          0   
2  Jules is, self declared, the most useless pers...       0          0   
3  #137 is a SCI/FI thriller about a girl, Marla,...       0          0   
4  A bright and talented performer, Chelsea Johns...       0          0   

   Biography  Comedy  Crime  Drama  Family  Fantasy  ...  Horror  Music  \
0          0       1      0      0       0        0  ...       0      0   
1          0       0      0      0       0        0  ...       1      0   
2          0       1      0      0       0        0  

In [4]:
# Create a dataframe with the plot column tokenized and lowercased
tokenizeddf = data.copy()
tokenizeddf['Plot'] = tokenizeddf['Plot'].apply(lambda x: nltk.word_tokenize(x.lower()))

In [5]:
print(tokenizeddf.head())

                                    Title  \
0          #1 Cheerleader Camp (2010) (V)   
1                 #1 Serial Killer (2013)   
2  #1 at the Apocalypse Box Office (2015)   
3                             #137 (2011)   
4                              #30 (2013)   

                                                Plot  Action  Adventure  \
0  [when, they, 're, hired, to, work, at, a, chee...       0          0   
1  [years, of, seething, rage, against, the, raci...       0          0   
2  [jules, is, ,, self, declared, ,, the, most, u...       0          0   
3  [#, 137, is, a, sci/fi, thriller, about, a, gi...       0          0   
4  [a, bright, and, talented, performer, ,, chels...       0          0   

   Biography  Comedy  Crime  Drama  Family  Fantasy  ...  Horror  Music  \
0          0       1      0      0       0        0  ...       0      0   
1          0       0      0      0       0        0  ...       1      0   
2          0       1      0      0       0        0  

In [6]:
# Get the length of the longest plot
maxlen = tokenizeddf['Plot'].apply(len).max()

# Get the set of all words in the plot column
wordset = set()
for plot in tokenizeddf['Plot']:
    wordset.update(plot)
# Get the number of unique words
numwords = len(wordset)
print("Number of unique words: ", numwords)

# Create a dictionary that maps words to integers
word2int = {word: i for i, word in enumerate(wordset)}

# Function to convert a list of words to a list of integers
def words2ints(words):
    # Run through each word in the list
    ints = []
    for word in words:
        # If the word is in the dictionary, add the integer to the list
        if word in word2int:
            ints.append(word2int[word])
    return ints

Number of unique words:  515729


In [7]:
# Convert the plot column to a list of integers
tokenizeddf['Plot'] = tokenizeddf['Plot'].apply(words2ints)

print(tokenizeddf.head())

# Pad the plot column with zeros to make all plots the same length
def pad_plot(plot):
    return plot + [0] * (maxlen - len(plot))

tokenizeddf['Plot'] = tokenizeddf['Plot'].apply(pad_plot)

                                    Title  \
0          #1 Cheerleader Camp (2010) (V)   
1                 #1 Serial Killer (2013)   
2  #1 at the Apocalypse Box Office (2015)   
3                             #137 (2011)   
4                              #30 (2013)   

                                                Plot  Action  Adventure  \
0  [488374, 93730, 21346, 418143, 475418, 396012,...       0          0   
1  [374574, 96524, 156151, 12143, 247429, 48943, ...       0          0   
2  [193678, 456847, 357459, 224798, 371110, 35745...       0          0   
3  [296323, 501868, 456847, 405264, 156958, 33526...       0          0   
4  [405264, 116557, 149640, 199627, 130841, 35745...       0          0   

   Biography  Comedy  Crime  Drama  Family  Fantasy  ...  Horror  Music  \
0          0       1      0      0       0        0  ...       0      0   
1          0       0      0      0       0        0  ...       1      0   
2          0       1      0      0       0        0  

In [8]:
# Make a dataframe with just title, plot, and action columns
# Going to try this out first on just action movies
actiondf = tokenizeddf[['Title', 'Plot', 'Action']]
print(actiondf.head())

# Encode the action column with LabelEncoder
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
actiondf["Action"] = le.fit_transform(actiondf["Action"])

                                    Title  \
0          #1 Cheerleader Camp (2010) (V)   
1                 #1 Serial Killer (2013)   
2  #1 at the Apocalypse Box Office (2015)   
3                             #137 (2011)   
4                              #30 (2013)   

                                                Plot  Action  
0  [488374, 93730, 21346, 418143, 475418, 396012,...       0  
1  [374574, 96524, 156151, 12143, 247429, 48943, ...       0  
2  [193678, 456847, 357459, 224798, 371110, 35745...       0  
3  [296323, 501868, 456847, 405264, 156958, 33526...       0  
4  [405264, 116557, 149640, 199627, 130841, 35745...       0  


C:\Users\zkand\AppData\Local\Temp\ipykernel_53880\2112378440.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actiondf["Action"] = le.fit_transform(actiondf["Action"])


In [10]:
# Create a Dataset class for the actiondata
class GenreDataset(torch.utils.data.Dataset):
    def __init__(self, data, genre):
        self.plot = data["Plot"].values
        self.genre = data[genre].values

    def __len__(self):
        return len(self.plot)
    
    def __getitem__(self, i):
        plot = self.plot[i]
        genre = self.genre[i]
        return torch.tensor(plot, dtype=torch.long), torch.tensor(genre, dtype=torch.float)

In [ ]:
# Train test split the actiondf
actiontrain, actiontest = train_test_split(actiondf, test_size=0.2, random_state=42)
    
# Create a GenreDataset object for the actiondata
actiontraindataset = GenreDataset(actiontrain, "Action")
actiontestdataset = GenreDataset(actiontest, "Action")

# Create a DataLoader for the actiondataset
actiontrainloader = torch.utils.data.DataLoader(actiontraindataset, batch_size=32, shuffle=True)
actiontestloader = torch.utils.data.DataLoader(actiontestdataset, batch_size=32, shuffle=False)

In [11]:
# Create the RNN to classify the plots as action or not
class GenreRNN(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_size):
        super(GenreRNN, self).__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embedding_dim)
        self.rnn = torch.nn.RNN(embedding_dim, hidden_dim, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, output_size)
        self.sigmoid = torch.nn.Sigmoid()
    
    def forward(self, x):
        x = self.embedding(x)
        h0 = torch.zeros(1, x.size(0), hidden).to(x.device)
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])        
        return out

# Set the hyperparameters
embed = 128
hidden = 128
output = 2

# Create the model
actionmodel = GenreRNN(numwords, embed, hidden, output)

From here, we'll train an RNN for each genre.

In [20]:
# Function to train a model
def train_genre_rnn(genre, epochs, embed, hidden, output):
    genremodel = GenreRNN(numwords, embed, hidden, output)
    # Extract the relevant columns from the dataframe
    genredf = tokenizeddf[['Title', 'Plot', genre]]
    # Encode the genre column
    genredf[genre] = le.fit_transform(genredf[genre])
    # Train test split the data
    # We won't use the test data in this function
    genretrain, genretest = train_test_split(genredf, test_size=0.3, random_state=42)
    # Create a GenreDataset object
    genretraindataset = GenreDataset(genretrain, genre)
    # Create a DataLoader
    genretrainloader = torch.utils.data.DataLoader(genretraindataset, batch_size=32, shuffle=True)

    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(genremodel.parameters(), lr=0.001)

    # Set the device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Send the model to the device
    genremodel.to(device)

    # Train the model
    num_epochs = epochs
    for epoch in range(num_epochs):
        epoch_loss = 0
        genremodel.train()
        for plots, genres in genretrainloader:
            # Send the data to the device
            plots = plots.to(device)
            genres = genres.to(device)
            outputs = genremodel(plots)
            loss = criterion(outputs, genres.long())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

    # Return the model
    return genremodel

In [13]:
# Save a model
def save_model(model, genre):
    if not os.path.exists("models/rnn"):
        os.makedirs("models/rnn")
    torch.save(model.state_dict(), f"models/rnn/{genre}model.pth")

In [14]:
# Get all the genres
genres = data.columns[2:]
print(genres)

Index(['Action', 'Adventure', 'Biography', 'Comedy', 'Crime', 'Drama',
       'Family', 'Fantasy', 'History', 'Horror', 'Music', 'Musical', 'Mystery',
       'Romance', 'Sci-Fi', 'Sport', 'Thriller', 'War', 'Western'],
      dtype='object')


In [28]:
# For each genre, check if there's already a model
# If not, train a model and save it
for genre in genres:
    if not os.path.exists(f"models/rnn/{genre}model.pth"):
        print(f"Training model for {genre}")
        model = train_genre_rnn(genre, 5, 128, 128, 2)
        save_model(model, genre)
    else:
        print(f"Model for {genre} already exists")

Model for Action already exists
Model for Adventure already exists
Model for Biography already exists
Model for Comedy already exists
Model for Crime already exists
Model for Drama already exists
Model for Family already exists
Model for Fantasy already exists
Training model for History


C:\Users\zkand\AppData\Local\Temp\ipykernel_53880\797772229.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genredf[genre] = le.fit_transform(genredf[genre])


Epoch 1/5, Loss: 0.6839067935943604
Epoch 2/5, Loss: 0.6818390488624573
Epoch 3/5, Loss: 0.6849406957626343
Epoch 4/5, Loss: 0.6859745383262634
Epoch 5/5, Loss: 0.6828728914260864
Training model for Horror


C:\Users\zkand\AppData\Local\Temp\ipykernel_53880\797772229.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genredf[genre] = le.fit_transform(genredf[genre])


Epoch 1/5, Loss: 0.6000033020973206
Epoch 2/5, Loss: 0.6000033020973206
Epoch 3/5, Loss: 0.626711905002594
Epoch 4/5, Loss: 0.6178090572357178
Epoch 5/5, Loss: 0.617808997631073
Training model for Music


...

Epoch 1/5, Loss: 0.8748595714569092
Epoch 2/5, Loss: 0.8568382859230042
Epoch 3/5, Loss: 0.8027744889259338
Epoch 4/5, Loss: 0.8568384051322937
Epoch 5/5, Loss: 0.874859631061554
Training model for War


C:\Users\zkand\AppData\Local\Temp\ipykernel_53880\797772229.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genredf[genre] = le.fit_transform(genredf[genre])


Epoch 1/5, Loss: 0.4824155867099762
Epoch 2/5, Loss: 0.42449700832366943
Epoch 3/5, Loss: 0.42449700832366943
Epoch 4/5, Loss: 0.453456312417984
Epoch 5/5, Loss: 0.42449700832366943
Training model for Western


C:\Users\zkand\AppData\Local\Temp\ipykernel_53880\797772229.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genredf[genre] = le.fit_transform(genredf[genre])


Epoch 1/5, Loss: 0.7799854874610901
Epoch 2/5, Loss: 0.7981731295585632
Epoch 3/5, Loss: 0.7981731295585632
Epoch 4/5, Loss: 0.7799856066703796
Epoch 5/5, Loss: 0.7799856066703796


In [29]:
# Function to get the genre scores for a plot
# Returns a dictionary with the genre as the key and the score (0-1) as the value
def predict_genres(plot):
    # Tokenize and pad the plot
    plot = nltk.word_tokenize(plot.lower())
    plot = words2ints(plot)
    plot = pad_plot(plot)
    plot = torch.tensor(plot, dtype=torch.long).unsqueeze(0)
    # Create a dictionary to store the scores
    scores = {}
    # For each genre, load the model and get the score
    for genre in genres:
        # Check if the model exists
        if not os.path.exists(f"models/rnn/{genre}model.pth"):
            print(f"Model for {genre} does not exist")
            continue
        print(f"Predicting {genre}")
        model = GenreRNN(numwords, 128, 128, 2)
        model.load_state_dict(torch.load(f"models/rnn/{genre}model.pth"))
        model.eval()
        with torch.no_grad():
            # Get the output of the model
            output = model(plot)
            # Convert the output to a probability
            prob = torch.nn.functional.softmax(output, dim=1)
            # Get the probability of the plot being in the genre
            score = prob[0][1].item()
            # Add the score to the dictionary
            scores[genre] = score
    return scores

In [30]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Make a prediction for a plot
plot = "Luke Skywalker joins forces with a Jedi Knight, a cocky pilot, a Wookiee and two droids to save the galaxy from the Empire's world-destroying battle station, while also attempting to rescue Princess Leia from the mysterious Darth Vader."
predictions = predict_genres(plot)
print(predictions)

Predicting Action
Predicting Adventure
Predicting Biography
Predicting Comedy
Predicting Crime
Predicting Drama
Predicting Family
Predicting Fantasy
Predicting History
Predicting Horror
Predicting Music
Predicting Musical
Predicting Mystery
Predicting Romance
Predicting Sci-Fi
Predicting Sport
Predicting Thriller
Predicting War
Predicting Western
{'Action': 0.3445218503475189, 'Adventure': 0.6334119439125061, 'Biography': 0.5937089920043945, 'Comedy': 0.5941818356513977, 'Crime': 0.5683135986328125, 'Drama': 0.5384088158607483, 'Family': 0.5249367952346802, 'Fantasy': 0.4671383202075958, 'History': 0.49430134892463684, 'Horror': 0.4511418044567108, 'Music': 0.48513540625572205, 'Musical': 0.5858104825019836, 'Mystery': 0.5334867238998413, 'Romance': 0.2626238465309143, 'Sci-Fi': 0.5303011536598206, 'Sport': 0.4862709045410156, 'Thriller': 0.597869336605072, 'War': 0.3459392786026001, 'Western': 0.5498623847961426}


Old stuff below here

In [263]:
# Make a dataframe that has only the title, plot, and action columns
actiondata = data[["Title", "Plot", "Action"]]
# Get the first 5 rows of the actiondata
print(actiondata.head())

# Encode the action column with LabelEncoder
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
actiondata["Action"] = le.fit_transform(actiondata["Action"])

# Get the length of the longest plot (for padding later on)
maxlen = actiondata["Plot"].apply(len).max()

# Get the set of all words in the plot column
wordset = set()
for plot in actiondata["Plot"]:
    wordset.update(plot)
# Get the number of unique words
numwords = len(wordset)
print("Number of unique words:", numwords)

# Create a dictionary that maps words to integers
word2int = {word: i for i, word in enumerate(wordset)}

# Function to convert a list of words to a list of integers
def words2ints(words):
    return [word2int[word] for word in words]

# Convert the plot column to a list of integers
actiondata["Plot"] = actiondata["Plot"].apply(words2ints)

# Get the first 5 rows of the actiondata
print(actiondata.head())

# Pad the plot column with zeros to make all plots the same length
def pad_plot(plot):
    return plot + [0] * (maxlen - len(plot))

actiondata["Plot"] = actiondata["Plot"].apply(pad_plot)

# Get the first 5 rows of the actiondata
print(actiondata.head())

KeyboardInterrupt: 

In [ ]:
# Create a Dataset class for the actiondata
class GenreDataset(torch.utils.data.Dataset):
    def __init__(self, data, genre):
        self.plot = data["Plot"].values
        self.genre = data[genre].values

    def __len__(self):
        return len(self.plot)
    
    def __getitem__(self, i):
        plot = self.plot[i]
        genre = self.genre[i]
        return torch.tensor(plot, dtype=torch.long), torch.tensor(genre, dtype=torch.float)
    
# Create a GenreDataset object for the actiondata
actiondataset = GenreDataset(actiondata, "Action")

# Create a DataLoader for the actiondataset
actionloader = torch.utils.data.DataLoader(actiondataset, batch_size=32, shuffle=True)

In [ ]:
# Create the RNN to classify the plots as action or not
class GenreRNN(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_size):
        super(GenreRNN, self).__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embedding_dim)
        self.rnn = torch.nn.RNN(embedding_dim, hidden_dim, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, output_size)
        self.sigmoid = torch.nn.Sigmoid()
    
    def forward(self, x):
        x = self.embedding(x)
        h0 = torch.zeros(1, x.size(0), hidden).to(x.device)
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])        
        return out

# Set the hyperparameters
embed = 128
hidden = 128
output = 2

# Create the model
actionmodel = GenreRNN(numwords, embed, hidden, output)

print(actiondataset[0])

(tensor([10155, 15944, 14626,  ...,     0,     0,     0]), tensor(0.))


In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(actionmodel.parameters(), lr=0.001)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    epoch_loss = 0
    actionmodel.train()
    for plots, genres in actionloader:
        outputs = actionmodel(plots)
        loss = criterion(outputs, genres.long())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

Epoch 1/10, Loss: 0.38106486201286316
Epoch 2/10, Loss: 0.13124583661556244
Epoch 3/10, Loss: 0.08345924317836761
Epoch 4/10, Loss: 0.09120485931634903
Epoch 5/10, Loss: 0.6385515332221985
Epoch 6/10, Loss: 0.3833125829696655
Epoch 7/10, Loss: 0.07289722561836243
Epoch 8/10, Loss: 0.12186636030673981
Epoch 9/10, Loss: 0.3818072974681854
Epoch 10/10, Loss: 0.38340580463409424


In [ ]:
# Look at the first five plots and their predicted genres
actionmodel.eval()
with torch.no_grad():
    for plots, genres in actionloader:
        outputs = actionmodel(plots)
        _, preds = torch.max(outputs.data, 1)
        #print("Predictions:", preds)
        #print("Actual:", genres)
        print((preds == genres)[0])
        print((preds == genres)[0].sum())
        print((preds == genres)[0].sum().item())
        num_correct = (preds == genres).sum().item()
        print(num_correct, "correct out of", len(genres))
        break

tensor(True)
tensor(1)
1
29 correct out of 32


In [ ]:
# Evaluate the model
actionmodel.eval()
correct = 0
total = 0
with torch.no_grad():
    for plots, genres in actionloader:
        outputs = actionmodel(plots)
        _, predicted = torch.max(outputs.data, 1)
        total += genres.size(0)
        num_correct = (predicted == genres).sum().item()
        correct += num_correct
print(correct, total)
accuracy = correct / total
print(f"Accuracy: {accuracy}")

916 1000
Accuracy: 0.916
